In [72]:
import numpy as np
import random

In [73]:
class SequenceEncoder(object):
    """
    Helper class for encoding / decoding number sequences
    """
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, 10))
        for i, c in enumerate(C):
            x[i, c] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return x
    
    def encode_position(self, num_rows, index):
        x = np.zeros([num_rows])
        x[index] = 1
        return x
    
    def decode_position(self,position_sequence):
        return position_sequence.argmax()
    
    def return_subsequence(self, 
                           input_sequence, 
                           start_position_sequence, 
                           end_position_sequence):
        start = self.decode_position(start_position_sequence)
        end = self.decode_position(end_position_sequence)
        decoded_input = self.decode(input_seq)
        return decoded_input[start:end]

In [74]:
sequence_encoder = SequenceEncoder()

In [75]:
sequence = [1, 2, 3]
max_len = 20

In [76]:
input_seq = sequence_encoder.encode([1,2,3], max_len)

In [77]:
start = sequence_encoder.encode_position(max_len, 0)

In [78]:
end = sequence_encoder.encode_position(max_len, 3)

In [79]:
sequence_encoder.return_subsequence(input_seq, start, end)

array([1, 2, 3])

In [98]:

def generate_low_high_low_sequence(length, min_seglen=5, max_seglen=10):
    seq_before = [(random.randint(1,5)) for x in range(random.randint(min_seglen, max_seglen))]
    seq_during = [(random.randint(6,9)) for x in range(random.randint(min_seglen, max_seglen))]
    seq_after = [random.randint(1,5) for x in range(random.randint(min_seglen, max_seglen))]
    seq = seq_before + seq_during + seq_after

    # Pad it up to max len with 0's
    seq = seq + ([0] * (length - len(seq)))
    return [seq, len(seq_before), len(seq_before) + len(seq_during)-1]

In [99]:
x ,start, end = generate_low_high_low_sequence(20)

In [108]:
sequence_encoder.encode(x, 20)

array([[ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.

In [101]:
x[start]

8

In [102]:
x[end]

9

In [227]:
def generate_batch(batch_size, sequence_gen_fun, input_len=60, min_seg=5, max_seg=10):
    sequences = []
    start_indices = []
    end_indices = []
    seq_encoder = SequenceEncoder()
    for i in range(batch_size):
        seq, start, end = sequence_gen_fun(input_len, min_seg, max_seg)
        sequences.append(sequence_encoder.encode(seq, input_len))
        start_indices.append(sequence_encoder.encode_position(input_len, start))
        end_indices.append(sequence_encoder.encode_position(input_len, end))
        
    
    return np.stack(sequences), np.stack([np.stack(start_indices), np.stack(end_indices)], axis=-1)

In [228]:
x_batch, y_batch = generate_batch(10000,generate_low_high_low_sequence)

In [229]:
x_batch.shape

(10000, 60, 10)

In [230]:
y_batch.shape

(10000, 60, 2)

In [231]:
from keras.models import Model, Sequential
from keras.layers import LSTM, TimeDistributed, RepeatVector, Dropout, Dense, Activation, Input, Bidirectional
from keras.optimizers import Adam, SGD


In [232]:
num_indices = 2
input_len = 60
enc_input = Input(shape=(input_len, 10))
enc = LSTM(40, return_sequences=True)(enc_input)
dec = LSTM(40, return_sequences=True)(enc)
dec = TimeDistributed(Dense(units=2, activation='softmax'))(dec)
model = Model(inputs=[enc_input], outputs=[dec])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_24 (InputLayer)        (None, 60, 10)            0         
_________________________________________________________________
lstm_31 (LSTM)               (None, 60, 40)            8160      
_________________________________________________________________
lstm_32 (LSTM)               (None, 60, 40)            12960     
_________________________________________________________________
time_distributed_18 (TimeDis (None, 60, 2)             82        
Total params: 21,202.0
Trainable params: 21,202.0
Non-trainable params: 0.0
_________________________________________________________________


In [235]:
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

In [236]:
model.fit(x_batch, y_batch, epochs=20)

Epoch 1/20
10000/10000 [==============================] - 104s - loss: 0.2806 - acc: 0.4935   
Epoch 2/20
10000/10000 [==============================] - 102s - loss: 0.2481 - acc: 0.4957   
Epoch 3/20
10000/10000 [==============================] - 103s - loss: 0.2466 - acc: 0.4872   
Epoch 4/20
10000/10000 [==============================] - 102s - loss: 0.2446 - acc: 0.4974   
Epoch 5/20
10000/10000 [==============================] - 104s - loss: 0.2444 - acc: 0.5013   
Epoch 6/20
10000/10000 [==============================] - 102s - loss: 0.2443 - acc: 0.5029   
Epoch 7/20
10000/10000 [==============================] - 103s - loss: 0.2443 - acc: 0.5013   
Epoch 8/20
10000/10000 [==============================] - 102s - loss: 0.2442 - acc: 0.4961   
Epoch 9/20
10000/10000 [==============================] - 106s - loss: 0.2442 - acc: 0.5028   
Epoch 10/20
10000/10000 [==============================] - 105s - loss: 0.2442 - acc: 0.5008   
Epoch 11/20
10000/10000 [========================

KeyboardInterrupt: 

In [237]:
sequence_encoder.decode(x_batch[0])

array([5, 3, 3, 3, 4, 1, 2, 1, 2, 4, 9, 8, 6, 6, 8, 3, 4, 3, 3, 4, 1, 3, 3,
       3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [239]:
u = model.predict(x_batch[:1]).transpose()

In [244]:
np.argmax(u[1])

14

In [243]:
y_batch[0]

array([[ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 1.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  1.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,

In [207]:
i, j = sequence_encoder.encode_position(60, start), sequence_encoder.encode_position(60, end)

In [208]:
i

array([ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [209]:
j

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [215]:
np.stack([i,j], axis=-1)

array([[ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 1.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  1.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,

In [246]:
seq, start, end = generate_low_high_low_sequence(60)

In [248]:
start, end

(5, 9)

In [249]:
test = np.stack([sequence_encoder.encode(seq, 60)])

In [253]:
np.argmax(model.predict(test), axis=1)

array([[5, 9]])